In [3]:
import os, json
import pandas as pd
import numpy as np
import math
import datetime as dt
import glob

In [5]:
#read in data
path_to_csv = 'C:/Users/Mark Hansen/daily_data/grouped/proccessed_R/*.csv'
file_list = glob.glob(path_to_csv)
csv_list = []
for f in file_list:
    data = pd.read_csv(f)
    csv_list.append(data)
    
df = pd.concat(csv_list)


In [7]:
df.head(100)

,call_time,heading,id,latitude,longitude,predictable,route_id,run_id,seconds_since_report,bus_id,direction,day
0,2020-01-22T07:00:04Z,4,5761,33.798464,-118.290649,True,205,205800,8,5761_205800_2020-01-22,0,2020-01-22
1,2020-01-22T07:00:04Z,357,1545,33.882894,-118.236660,True,205,205800,8,1545_205800_2020-01-22,0,2020-01-22
2,2020-01-22T07:00:04Z,90,1552,33.785950,-118.282089,True,205,205811,35,1552_205811_2020-01-22,1,2020-01-22
3,2020-01-22T07:00:04Z,165,1508,33.901779,-118.238167,True,205,205811,8,1508_205811_2020-01-22,1,2020-01-22
4,2020-01-22T07:02:05Z,0,5761,33.806680,-118.290286,True,205,205800,47,5761_205800_2020-01-22,0,2020-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020-01-22T07:44:17Z,240,1508,33.865726,-118.275177,True,205,205811,49,1508_205811_2020-01-22,1,2020-01-22
96,2020-01-22T07:46:18Z,13,1538,33.767133,-118.310314,True,205,205800,7,1538_205800_2020-01-22,0,2020-01-22
97,2020-01-22T07:46:18Z,356,5761,33.858502,-118.234783,True,205,205800,34,5761_205800_2020-01-22,0,2020-01-22
98,2020-01-22T07:46:18Z,174,1545,33.924082,-118.239188,True,205,205811,7,1545_205811_2020-01-22,1,2020-01-22


In [8]:
#convert timestamps to datetime format
df['call_time'] = pd.to_datetime(df['call_time'])
#create report time variable (difference between the time the bus reported its coordinates and my ping)
df['report_time'] = df['call_time'] - pd.to_timedelta(df['seconds_since_report'], unit='s')

In [9]:
#sort by bus_id and report_time, calculate time deltas
df = df.sort_values(by=['bus_id', 'report_time'])
df['diff'] = df.groupby('bus_id')['report_time'].diff()

In [10]:
#if the ping response has the same report as before, the difference between report times is 0, thus useless
df = df[df['diff'].dt.total_seconds() != 0]

In [11]:
#haversine function (the earth is big blue sphere)
import numpy as np
def haversine(lon1, lat1, lon2, lat2, earth_radius=6367):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = earth_radius * c
    return km

In [12]:
#group and add prior report's lat long
dfg = df.groupby('bus_id')
df ['p_latitude'] = dfg['latitude'].shift(1)
df ['p_longitude'] = dfg['longitude'].shift(1)
#i think the NAs are puttings 0,0 coordinates into my distance calculations?
df = df.dropna(subset=['diff'])


In [13]:
#calculate distance in KM by applying haversine function to each pair
df['distance'] = df[['p_latitude', 'p_longitude', 'latitude','longitude']].apply(lambda x: haversine(x[1], x[0], x[3], x[2]), axis=1)

C:\Users\Mark Hansen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
#get hours
df['diff'] = df['diff'].dt.total_seconds()
df['diff'] = df['diff']/3600

C:\Users\Mark Hansen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Mark Hansen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
#convert KM to miles
df['distance'] = df['distance']/1.609

C:\Users\Mark Hansen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
#speed is distance over time
df['mph'] = df['distance']/df['diff']

C:\Users\Mark Hansen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
df.to_csv('C:/Users/Mark Hansen/daily_data/grouped/processed_speed/bus_speeds_3-03.csv')